In [5]:
import pandas as pd

In [65]:
df = pd.read_csv('gpt_query_data.csv',header=None,names=['query', 'fin_news', 'duration', 'recency', 'vague', 'search_query'])

In [66]:
df

,query,fin_news,duration,recency,vague,search_query
0,sebi bond platform,True,-1.0,Medium,False,SEBI bond platform details
1,wint wealth,True,-1.0,Low,False,Wint Wealth financial information
2,kkr private equity,True,-1.0,Medium,False,KKR private equity details
3,ev infra,True,30.0,Medium,True,ev infra financial and business news
4,rural banking in india,True,-1.0,Medium,False,Rural banking sector in India
...,...,...,...,...,...,...
406,yygggggggggg,False,NaN,NaN,NaN,NaN
407,inflation rate in india,True,30.0,High,False,Current inflation rate in India
408,budget,True,30.0,High,False,India's budget highlights
409,tata,True,30.0,Medium,True,Tata financial and business news


In [14]:
import boto3
iam = boto3.client('iam')
response = iam.list_roles()
sagemaker_roles = [role for role in response['Roles'] if 'SageMaker' in role['RoleName']]
sagemaker_roles[0]['RoleName']

'AmazonSageMaker-ExecutionRole-20231030T210397'

In [15]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [16]:
import pandas as pd
import numpy as np
import json

import re
from transformers import AutoTokenizer
from random import randint
import sys
sys.path.append("utils")
from pack_dataset import pack_dataset
from datasets import Dataset

import requests

In [17]:
model_id = "teknium/OpenHermes-2.5-Mistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Generating training data

In [21]:
from datetime import datetime

In [23]:
datetime.now().strftime('%d/%m/%y')

'18/01/24'

In [25]:
from datetime import timedelta

In [38]:
dt = (datetime.now() - timedelta(days = np.random.randint(730))).strftime('%d/%m/%y')

In [24]:
datetime.date?

Docstring: Return date object with same year, month and day.
Type:      method_descriptor

In [249]:
system_prompt = f'''
You are an expert in taking user queries on an indian financial and business website and extracting certain attributes like below. Generate the attributes in a csv safe format with columns separated by comma. The last attribute of web search query is extremely important, and should directly result in relevant financial results when applicable. If the query is too vague to get only financial results, then append it with ‘financial and business news’. Today is {dt} (%d/%m/%y). use MMMM d yyyy when applicable in web search query
is_financial_query, days_to_filter(-1 for timeless), recency_importance(High for highly volatile, medium for moderate volatility like regulations or industry related, and low for educational and guides), query_too_vague_for_finance, web_search_query
Generate the responses for the below user query. No preamble or postamble
'''

In [250]:
def generate_prompt_with_date(date):
    return f'''
You are an expert in taking user queries on an indian financial and business website and extracting certain attributes like below. You generate the attributes in a csv safe format with columns separated by comma. The final attribute of web search query is extremely important, and should directly result in relevant results when the query has any financial or business connection. If the entity or query is too vague to get only financial or business results, then append it with ‘financial and business news’. Today is {date}. When using in web query, then use %d/%m/%y
- is_fin_business_query, recency_importance (High for very volatile, medium for slowly changing stuff like regulations or industry related, and low for very less changing like educational and guides), vague (too vague to generate financial or business results), web_search_query (ideal web search query which will result in most relevant financial results if applicable)
Generate the responses for the below user query. No preamble or postamble.
'''

In [251]:
message_template = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": f" '|query_start|\n'{''}'\n|query_end|'\n"}]

OUTPUT_TOKEN_LIMIT = 256 # set based on the distribution of completion tokens from gpt4
INSTRUCTION_TOKENS = len(tokenizer.apply_chat_template(message_template, add_generation_prompt=True))
BUFFER_TOKENS = 10

In [252]:
INSTRUCTION_TOKENS

233

In [253]:
tokenizer.decode(tokenizer.apply_chat_template(message_template, add_generation_prompt=True))

"<|im_start|> system\n\nYou are an expert in taking user queries on an indian financial and business website and extracting certain attributes like below. Generate the attributes in a csv safe format with columns separated by comma. The last attribute of web search query is extremely important, and should directly result in relevant financial results when applicable. If the query is too vague to get only financial results, then append it with ‘financial and business news’. Today is 19/06/23 (%d/%m/%y). use %d/%m/%y when applicable in web search query\nis_financial_query, days_to_filter(-1 for timeless), recency_importance(High for highly volatile, medium for moderate volatility like regulations or industry related, and low for educational and guides), query_too_vague_for_finance, web_search_query\nGenerate the responses for the below user query. No preamble or postamble\n<|im_end|> \n<|im_start|> user\n '|query_start|\n''\n|query_end|'\n<|im_end|> \n<|im_start|> assistant\n"

In [143]:
INSTRUCTION_TOKENS

228

In [82]:
df = df.astype(str)
df = df.replace('nan','')
df['duration'] = df['duration'].apply(lambda x: str(int(float(x))) if x else '')

In [256]:
# df['response'] = df['fin_news'] + ',' + df['duration'] + ',' + df['recency'] + ',' + df['vague'] + ',' + df['search_query']

In [254]:
df['response'] = df['fin_news'] + ',' + df['recency'] + ',' + df['vague'] + ',' + df['search_query']

In [257]:
df['random_date'] = [(datetime.now() - timedelta(days = np.random.randint(730))).strftime('%d/%m/%y') for _ in range(len(df))]

In [261]:
# def get_readable_date(date):
#     date_obj = datetime.strptime(date, "%d/%m/%y")
#     return date_obj.strftime("%B %d %Y")

In [262]:
# df['readable_date'] = df['random_date'].apply(lambda x: get_readable_date(x))

In [258]:
df['response'] = df.apply(lambda x: x['response'].replace('January 18 2024', x['random_date']),axis =1)

In [259]:
df.iloc[281].response

'True,High,False,Wipro share price on 15/10/23'

In [260]:
df.iloc[281]

query                                   wipro share price live
fin_news                                                  True
duration                                                     1
recency                                                   High
vague                                                    False
search_query              Wipro share price on January 18 2024
response         True,High,False,Wipro share price on 15/10/23
len                                                         22
random_date                                           15/10/23
readable_date                                 November 10 2022
Name: 281, dtype: object

In [255]:
df.sample(30)

,query,fin_news,duration,recency,vague,search_query,response,len,random_date,readable_date
51,bajaj auto share price live,True,1,High,False,Bajaj Auto share price on January 18 2024,"True,High,False,Bajaj Auto share price on Janu...",27,28/05/22,May 28 2022
363,share price today,True,1,High,False,Share prices today in Indian market,"True,High,False,Share prices today in Indian m...",17,07/01/24,January 07 2024
389,ntpc share price live,True,1,High,False,NTPC share price live on January 18 2024,"True,High,False,NTPC share price live on Janua...",21,21/09/23,September 21 2023
195,supreme court,True,30,Medium,True,Supreme court financial and business news,"True,Medium,True,Supreme court financial and b...",13,20/04/23,April 20 2023
353,whats p2p,True,-1,Low,False,What is P2P lending,"True,Low,False,What is P2P lending",9,16/01/23,January 16 2023
374,stock market today,True,1,High,False,Indian stock market performance today,"True,High,False,Indian stock market performanc...",18,22/05/23,May 22 2023
63,day trade,True,1,High,False,Day trading strategies,"True,High,False,Day trading strategies",9,15/07/22,July 15 2022
165,reliance,True,30,Medium,False,Reliance Industries financial and business news,"True,Medium,False,Reliance Industries financia...",8,12/10/22,October 12 2022
234,haryana,True,365,Medium,True,Haryana financial and business news,"True,Medium,True,Haryana financial and busines...",7,17/09/23,September 17 2023
297,fintech,True,-1,Medium,False,Fintech industry trends in India,"True,Medium,False,Fintech industry trends in I...",7,18/11/23,November 18 2023


In [264]:
def format_text_response_as_prompt(train_row):
    query=train_row['query']
    messages = [{"role": "system", "content": generate_prompt_with_date(train_row['random_date'])},
                {"role": "user", "content": f" '|query_start|\n'{query}'\n|query_end|'\n"}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    prompt = context_prompt + train_row['response']
    prompt = re.sub(r'\n+','\n',prompt)
    return prompt

In [265]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Creating dataset

In [282]:
dataset = Dataset.from_pandas(df)

In [283]:
# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_text_response_as_prompt(sample)}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(template_dataset)

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

In [288]:
dataset[132]

{'query': 'bojack horseman',
 'fin_news': 'False',
 'duration': '',
 'recency': '',
 'vague': '',
 'search_query': '',
 'response': 'False,,,',
 'len': 15,
 'random_date': '17/06/23',
 'readable_date': 'October 19 2022',
 'text': "<|im_start|> system\nYou are an expert in taking user queries on an indian financial and business website and extracting certain attributes like below. You generate the attributes in a csv safe format with columns separated by comma. The final attribute of web search query is extremely important, and should directly result in relevant results when the query has any financial or business connection. If the entity or query is too vague to get only financial or business results, then append it with ‘financial and business news’. Today is 17/06/23. When using in web query, then use %d/%m/%y\n- is_fin_business_query, recency_importance (High for very volatile, medium for slowly changing stuff like regulations or industry related, and low for very less changing lik

In [285]:
dataset.select(range(2)).features

{'query': Value(dtype='string', id=None),
 'fin_news': Value(dtype='string', id=None),
 'duration': Value(dtype='string', id=None),
 'recency': Value(dtype='string', id=None),
 'vague': Value(dtype='string', id=None),
 'search_query': Value(dtype='string', id=None),
 'response': Value(dtype='string', id=None),
 'len': Value(dtype='int64', id=None),
 'random_date': Value(dtype='string', id=None),
 'readable_date': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [267]:
# tokenize dataset
dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
)
# chunk dataset
lm_dataset = pack_dataset(dataset, chunk_length=384) # We use 384 as the maximum length for packing
print(f"Total number of samples: {len(lm_dataset)}")

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

Map:   0%|          | 0/411 [00:00<?, ? examples/s]

Chunking dataset into chunks of 384 tokens.


Map:   0%|          | 0/411 [00:00<?, ? examples/s]

Total number of samples: 293
Total number of samples: 293


In [280]:
lm_dataset.select(range(1))

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})

### Getting fine tune id

In [268]:
with open('words.txt','r') as f:
    words = f.read()

In [269]:
words = words.split('\n')

In [270]:
finetune_id = ''.join(np.random.choice(words, 3))

In [271]:
finetune_id = 'search_query_' + finetune_id

In [272]:
finetune_id

'search_query_KiwiHammockEscalator'

In [273]:
finetune_dataset_config = {'finetune_id': finetune_id,
                          'date': datetime.strftime(datetime.today(),'%Y-%m-%d'),
                          'num_datapoints': len(df),
                            'data_source': 'gpt4',
                          'prompt': system_prompt}

### Save data to s3

In [274]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/fine_tuning_datasets/{finetune_dataset_config["date"]}-{finetune_dataset_config["finetune_id"]}'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/293 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-01-19-search_query_KiwiHammockEscalator


### Hyperparams

In [290]:
from huggingface_hub import HfFolder


# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                             # pre-trained model
  'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
  'num_train_epochs': 6,                            # number of training epochs
  'per_device_train_batch_size': 5,                 # batch size for training
  'gradient_accumulation_steps': 5,                 # Number of updates steps to accumulate
  'gradient_checkpointing': True,                   # save memory but slower backward pass
  'bf16': True,                                     # use bfloat16 precision
  'tf32': True,                                     # use tf32 precision
  'learning_rate': 5e-5,                            # learning rate
  'max_grad_norm': 0.3,                             # Maximum norm (for gradient clipping)
  'warmup_ratio': 0.03,                             # warmup ratio
  "lr_scheduler_type":"cosine_with_restarts",                   # learning rate scheduler
  'save_strategy': "epoch",                         # save strategy for checkpoints
  "logging_steps": 5,                              # log every x steps
  'merge_adapters': True,                           # wether to merge LoRA into the model (needs more memory)
  'use_flash_attn': True,                           # Whether to use Flash Attention
  'output_dir': '/tmp/run'                         # output directory, where to save assets during training
                                                    # could be used for checkpointing. The final trained
                                                    # model will always be saved to s3 at the end of training
}

if HfFolder.get_token() is not None:
    hyperparameters['hf_token'] = HfFolder.get_token() # huggingface token to access gated models, e.g. llama 2

In [291]:
from sagemaker.huggingface import HuggingFace

# define Training Job Name
job_name = f'huggingface-qlora-{hyperparameters["model_id"].replace("/","-").replace(".","-")}-{finetune_dataset_config["finetune_id"]}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_qlora.py',    # train script
    source_dir           = 'utils/',      # directory which includes all the files needed for training
    instance_type        = 'ml.g5.4xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    max_run              = 1*60*60,        # maximum runtime in seconds (days * hours * minutes * seconds)
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 100,               # the size of the EBS volume in GB
    transformers_version = '4.28',            # the transformers version used in the training job
    pytorch_version      = '2.0',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
    disable_output_compression = True         # not compress output to save training time and cost
)

### Training Job

In [292]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-qlora-teknium-OpenHermes-2--2024-01-19-04-23-49-721


2024-01-19 04:23:51 Starting - Starting the training job...
2024-01-19 04:24:11 Starting - Preparing the instances for training......
2024-01-19 04:25:12 Downloading - Downloading input data...
2024-01-19 04:25:37 Downloading - Downloading the training image.....................
2024-01-19 04:29:03 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-01-19 04:29:58,796 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-01-19 04:29:58,815 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-01-19 04:29:58,823 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-01-19 04:29:58,825 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-01-19 04:30:00,145 sagemaker-training-toolkit INFO     Installing depe

### Deployment

In [293]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.1.0",
  session=sess,
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

INFO:sagemaker.image_uris:Defaulting to only available Python version: py39
INFO:sagemaker.image_uris:Defaulting to only supported image scope: gpu.


llm image uri: 763104351884.dkr.ecr.ap-south-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04


In [294]:
import boto3
import os

# Initialize a boto3 S3 client
s3 = boto3.client('s3')

In [295]:
model_s3_path = huggingface_estimator.model_data["S3DataSource"]["S3Uri"]

In [296]:
model_s3_path

's3://sagemaker-ap-south-1-005418323977/huggingface-qlora-teknium-OpenHermes-2--2024-01-19-04-23-49-721/output/model/'

In [407]:
import json
from sagemaker.huggingface import HuggingFaceModel
model_s3_path = huggingface_estimator.model_data["S3DataSource"]["S3Uri"]
# sagemaker config
instance_type = "ml.g5.xlarge"
number_of_gpu = 1
health_check_timeout = 300

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(320), # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(384), # Max length of the generation (including input text)
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data={'S3DataSource':{'S3Uri': model_s3_path,'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  env=config
)

In [408]:
finetune_id

'search_query_KiwiHammockEscalator'

In [422]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
    endpoint_name = f"OpenHermes-{finetune_id.replace('_','-')}",
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)

INFO:sagemaker:Creating model with name: huggingface-pytorch-tgi-inference-2024-01-19-09-26-14-961
INFO:sagemaker:Creating endpoint-config with name OpenHermes-search-query-KiwiHammockEscalator
INFO:sagemaker:Creating endpoint with name OpenHermes-search-query-KiwiHammockEscalator


---------!

In [423]:
llm.endpoint_name

'OpenHermes-search-query-KiwiHammockEscalator'

### Test endpoint

In [424]:
smr = sess.boto_session.client("sagemaker-runtime")

In [437]:
parameters = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.7,
    "max_new_tokens": 512,
    "repetition_penalty": 1.02,
    "stop": ["###", "</s>", tokenizer.eos_token],
}

In [426]:
today = datetime.now().strftime('%d/%m/%y')

In [427]:
today

'19/01/24'

In [428]:
system_prompt = f'''
You are an expert in taking user queries on an indian financial and business website and extracting certain attributes like below. You generate the attributes in a csv safe format with columns separated by comma. The final attribute of web search query is extremely important, and should directly result in relevant results when the query has any financial or business connection. If the entity or query is too vague to get only financial or business results, then append it with ‘financial and business news’. Today is {today}. When using in web query, then use %d/%m/%y
- is_fin_business_query, recency_importance (High for very volatile, medium for slowly changing stuff like regulations or industry related, and low for very less changing like educational and guides), vague (too vague to generate financial or business results), web_search_query (ideal web search query which will result in most relevant financial results if applicable)
Generate the responses for the below user query. No preamble or postamble.
'''

In [429]:
system_prompt

'\nYou are an expert in taking user queries on an indian financial and business website and extracting certain attributes like below. You generate the attributes in a csv safe format with columns separated by comma. The final attribute of web search query is extremely important, and should directly result in relevant results when the query has any financial or business connection. If the entity or query is too vague to get only financial or business results, then append it with ‘financial and business news’. Today is 19/01/24. When using in web query, then use %d/%m/%y\n- is_fin_business_query, recency_importance (High for very volatile, medium for slowly changing stuff like regulations or industry related, and low for very less changing like educational and guides), vague (too vague to generate financial or business results), web_search_query (ideal web search query which will result in most relevant financial results if applicable)\nGenerate the responses for the below user query. No

In [430]:
def format_query_for_prompt(query):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": f"|query_start|\n'{query}'\n|query_end|'\n"}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    prompt = re.sub(r'\n+','\n',context_prompt)
    return prompt

In [431]:
format_query_for_prompt('sensex today')

"<|im_start|> system\nYou are an expert in taking user queries on an indian financial and business website and extracting certain attributes like below. You generate the attributes in a csv safe format with columns separated by comma. The final attribute of web search query is extremely important, and should directly result in relevant results when the query has any financial or business connection. If the entity or query is too vague to get only financial or business results, then append it with ‘financial and business news’. Today is 19/01/24. When using in web query, then use %d/%m/%y\n- is_fin_business_query, recency_importance (High for very volatile, medium for slowly changing stuff like regulations or industry related, and low for very less changing like educational and guides), vague (too vague to generate financial or business results), web_search_query (ideal web search query which will result in most relevant financial results if applicable)\nGenerate the responses for the b

In [438]:
request = {"inputs": format_query_for_prompt('apple'), "parameters": parameters, "stream": True}

In [433]:
import time

In [435]:
import io
import json

class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line) + 1
                full_line = line[:-1].decode("utf-8")
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["token"]["text"]
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

In [436]:
t = time.time()
resp = smr.invoke_endpoint_with_response_stream(
    EndpointName=f"OpenHermes-{finetune_id.replace('_','-')}",
    Body=json.dumps(request),
    ContentType="application/json",
)
print(f'done in {time.time() - t} seconds')
# k = resp['Body'].read()
for token in TokenIterator(resp["Body"]):
    print(token, end="")

done in 0.05632305145263672 seconds
True,Medium,True,Apple financial and business news<|im_end|>

In [440]:
from flask import Response

In [445]:
def completion(query):
    request = {"inputs": format_query_for_prompt(query), "parameters": parameters, "stream": True}
    def stream(request):
        t = time.time()
        resp = smr.invoke_endpoint_with_response_stream(
            EndpointName=f"OpenHermes-ApricotTangerineScallop-TGI",
            Body=json.dumps(request),
            ContentType="application/json",
        )
        print(f'done in {time.time() - t} seconds')
        # k = resp['Body'].read()
        for token in TokenIterator(resp["Body"]):
            yield token
    return Response(stream(request), mimetype='text/event-stream')

In [453]:
boto3.__version__

'1.34.15'

In [452]:
completion('banana')

<Response streamed [200 OK]>

In [451]:
for i in a.:
    print(a)

TypeError: 'Response' object is not iterable

In [419]:
t = time.time()
resp = smr.invoke_endpoint(
    EndpointName=f"OpenHermes-{finetune_id.replace('_','-')}",
    Body=json.dumps(request),
    ContentType="application/json",
)
print(f'done in {time.time() - t} seconds')
k = resp['Body'].read()

k

done in 0.978435754776001 seconds


b'[{"generated_text":"True,Medium,True,Apple financial and business news"}]'

In [420]:
json.loads(k)[0]['generated_text'].split(',')

['True', 'Medium', 'True', 'Apple financial and business news']

In [421]:
llm.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: OpenHermes-search-query-KiwiHammockEscalator
INFO:sagemaker:Deleting endpoint with name: OpenHermes-search-query-KiwiHammockEscalator


In [352]:
# llm.delete_endpoint()